## Collaborative Filtering

In [1]:
# Import relevant libraries
import pandas as pd
import numpy as np
import math

In [2]:
def read_ratings_xlsx(path, sheet_name=0):
    df = pd.read_excel(path, sheet_name, header=0)
    first_col = df.columns[0]
    df = df.rename(columns={first_col: "critic"})

    df['critic'] = df['critic'].astype(str).str.strip()
    df = df.set_index('critic')
    df.columns = [str(c).strip() for c in df.columns]
    df = df.apply(pd.to_numeric, errors='coerce')

    result = {}
    for critic, row in df.iterrows():
        movies = {}
        for movie, val in row.dropna().items(): 
            movies[movie] = float(val)
        result[critic] = movies

    return result, df.columns.to_list(), df

# Get the critics' ratings and movie list
critiques, movies, df = read_ratings_xlsx("data/movie_critique.xlsx")

# Get the list of movies rated by Lisa Rose
critiques['Lisa Rose']

{'Lady': 2.5,
 'Snakes': 3.5,
 'Luck': 3.0,
 'Superman': 3.5,
 'Dupree': 2.5,
 'Night': 3.0}

### Simple Distance Functions

In [3]:
def sim_distanceManhattan(person1, person2):
    distance = 0
    for critique1 in person1:
        if critique1 in person2:
            distance += abs(person1[critique1] - person2[critique1])
    return distance

# Get the Manhattan distance between Lisa Rose and Gene Seymour
sim_distanceManhattan(critiques['Lisa Rose'], critiques['Gene Seymour'])

4.5

In [4]:
def sim_distanceEuclidienne(person1, person2):
    distance = 0
    for critique1 in person1:
        if critique1 in person2:
            distance += (person1[critique1] - person2[critique1]) ** 2
    return distance ** 0.5

# Get the Euclidean distance between Lisa Rose and Gene Seymour
sim_distanceEuclidienne(critiques['Lisa Rose'], critiques['Gene Seymour'])

2.3979157616563596

In [5]:
def computeNearestNeighbor(nouveauCritique, critiques):
    distances=[]
    for critique in critiques:
        if critique!=nouveauCritique:
            distance=sim_distanceManhattan(critiques[critique], critiques[nouveauCritique])
            distances.append((distance, critique))
    distances.sort()
    return distances

# Find the nearest neighbors for Lisa Rose
computeNearestNeighbor('Lisa Rose', critiques)

[(1.5, 'Michael Phillips'),
 (2.0, 'Claudia Puig'),
 (2.5, 'Anne'),
 (3.0, 'Mick Lasalle'),
 (3.0, 'Toby'),
 (3.5, 'Jack Matthews'),
 (4.5, 'Gene Seymour')]

In [6]:
def recommendNearestNeighbor(nouveauCritique, critiques):
    distances = computeNearestNeighbor(nouveauCritique, critiques)
    nearestNeighbor = critiques[distances[0][1]]
    recommendations = []

    for critique in nearestNeighbor:
        if critique not in critiques[nouveauCritique]:
            recommendations.append((critique, nearestNeighbor[critique]))

    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)

    return recommendations

# Get movie recommendations for Lisa Rose
recommendNearestNeighbor('Lisa Rose', critiques)

[]

In [7]:
computeNearestNeighbor('Toby', critiques)

[(1.0, 'Anne'),
 (2.0, 'Michael Phillips'),
 (2.5, 'Claudia Puig'),
 (2.5, 'Mick Lasalle'),
 (3.0, 'Lisa Rose'),
 (4.0, 'Jack Matthews'),
 (4.5, 'Gene Seymour')]

In [8]:
recommendNearestNeighbor('Toby', critiques)

[('Luck', 4.0), ('Lady', 1.5)]

In [9]:
def computeNearestNeighborEuclidienne(nouveauCritique, critiques):
    distances=[]
    for critique in critiques:
        if critique!=nouveauCritique:
            distance=sim_distanceEuclidienne(critiques[critique], critiques[nouveauCritique])
            distances.append((distance, critique))
    distances.sort()
    return distances

# Find the nearest neighbors for Toby using Euclidean distance
computeNearestNeighborEuclidienne('Toby', critiques)

[(1.0, 'Anne'),
 (1.5, 'Mick Lasalle'),
 (1.5811388300841898, 'Michael Phillips'),
 (1.8027756377319946, 'Claudia Puig'),
 (1.8708286933869707, 'Lisa Rose'),
 (2.7386127875258306, 'Jack Matthews'),
 (2.8722813232690143, 'Gene Seymour')]

Notably, when Euclidian distance is used instead, the ranking of distances changes slightly.

In [10]:
def recommendNearestNeighborEuclidienne(nouveauCritique, critiques):
    distances = computeNearestNeighborEuclidienne(nouveauCritique, critiques)
    nearestNeighbor = critiques[distances[0][1]]
    recommendations = []

    for critique in nearestNeighbor:
        if critique not in critiques[nouveauCritique]:
            recommendations.append((critique, nearestNeighbor[critique]))

    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return recommendations

# Get movie recommendations for Toby using Euclidean distance
recommendNearestNeighborEuclidienne('Toby', critiques)

[('Luck', 4.0), ('Lady', 1.5)]

BUT - the closest critic remains the same.

In [11]:
def BestRecommendNearestNeighbor(recommendations):
    # Simple function to return the best recommendation given the above functions
    name, score = recommendations[0]
    return (name, round(float(score), 2))

# Get the best recommendation for Toby using Manhattan distance
BestRecommendNearestNeighbor(recommendNearestNeighbor('Toby', critiques))

('Luck', 4.0)

### Weighted Recommendations

#### Manhattan and Euclidian Recommendations

In [12]:
def Bestrecommend(nouveauCritique, critiques, movies):
    # Weighted recommendation function using Manhattan distance
    distances = computeNearestNeighbor(nouveauCritique, critiques)
    globalScores = {}
    for movie in movies:
        if movie not in critiques[nouveauCritique]:
            scoreSum = 0
            weightSum = 0
            globalScores[movie] = 0
            for i in range(len(distances)):
                criticName = distances[i][1]
                if movie in critiques[criticName]:
                    score = critiques[criticName][movie]
                    weight = 1 / (1 + distances[i][0])
                    scoreSum += score * weight
                    weightSum += weight

            if weightSum > 0:
                globalScores[movie] = scoreSum / weightSum
                
    globalScores = sorted(globalScores.items(), key=lambda x: x[1], reverse=True)

    name, score = globalScores[0]
    return (name, round(float(score), 2))

In [13]:
def BestrecommendEuclidienne(nouveauCritique, critiques, movies):
    # Weighted recommendation function using Euclidean distance
    distances = computeNearestNeighborEuclidienne(nouveauCritique, critiques)

    globalScores = {}
    for movie in movies:
        if movie not in critiques[nouveauCritique]:
            scoreSum = 0
            weightSum = 0
            globalScores[movie] = 0
            for i in range(len(distances)):
                criticName = distances[i][1]
                if movie in critiques[criticName]:
                    score = critiques[criticName][movie]
                    weight = 1 / (1 + distances[i][0])
                    scoreSum += score * weight
                    weightSum += weight

            if weightSum > 0:
                globalScores[movie] = scoreSum / weightSum
    
    globalScores = sorted(globalScores.items(), key=lambda x: x[1], reverse=True)

    name, score = globalScores[0]
    return (name, round(float(score), 2))

#### Exponential Recommendation

In [14]:
def BestrecommendwithExp(nouveauCritique, critiques, movies):
    distances = computeNearestNeighbor(nouveauCritique, critiques)
    globalScores = {}
    for movie in movies:
        if movie not in critiques[nouveauCritique]:
            scoreSum = 0
            weightSum = 0
            globalScores[movie] = 0
            for i in range(len(distances)):
                criticName = distances[i][1]
                if movie in critiques[criticName]:
                    score = critiques[criticName][movie]
                    weight = np.exp(-distances[i][0])
                    scoreSum += score * weight
                    weightSum += weight

            if weightSum > 0:
                globalScores[movie] = scoreSum / weightSum


    globalScores = sorted(globalScores.items(), key=lambda x: x[1], reverse=True)
    
    name, score = globalScores[0]
    return (name, round(float(score), 2))

#### Pearson Recommendation

In [15]:
def pearson(person1, person2):
    sum_xy=0
    sum_x=0
    sum_y=0
    sum_x2=0
    sum_y2=0
    n=0
    for key in person1:
        if key in person2:
            n += 1
            x=person1[key]
            y=person2[key]
            sum_xy +=x*y
            sum_x += x
            sum_y += y
            sum_x2 += x**2
            sum_y2 += y**2
    if n == 0:
        return 0 
    denominator = math.sqrt(sum_x2 - (sum_x**2) / n) * math.sqrt(sum_y2 - (sum_y**2) / n)
    if denominator == 0:
        return 0
    else:
        return (sum_xy - (sum_x * sum_y) /n ) / denominator

def PearsonRecommend(nouveauCritique, critiques, movies):
    globalScores = {}
    for movie in movies:
        if movie not in critiques[nouveauCritique]:
            scoreSum = 0
            weightSum = 0
            globalScores[movie] = 0
            for critic in critiques:
                if movie in critiques[critic]:
                    score = critiques[critic][movie]
                    weight = pearson(critiques[critic], critiques[nouveauCritique])
                    scoreSum += score * weight
                    weightSum += np.abs(weight)

            if weightSum > 0:
                globalScores[movie] = scoreSum / weightSum
    
    globalScores = sorted(globalScores.items(), key=lambda x: x[1], reverse=True)

    name, score = globalScores[0]
    return (name, round(float(score), 2))

#### Cosine Recommendation

In [16]:
def cosine(person1, person2):
    sum_x2 = 0
    sum_y2 = 0
    sum_xy = 0
    for key in person1:
        if key in person2:
            x=person1[key]
            y=person2[key]
            sum_xy += x * y
            sum_x2 += x ** 2
            sum_y2 += y ** 2
    denominator = math.sqrt(sum_x2) * math.sqrt(sum_y2)
    if denominator == 0:
        return 0
    else:
        return sum_xy / denominator

def CosineRecommend(nouveauCritique, critiques, movies):
    globalScores = {}
    for movie in movies:
        if movie not in critiques[nouveauCritique]:
            scoreSum = 0
            weightSum = 0
            globalScores[movie] = 0
            for critic in critiques:
                if movie in critiques[critic]:
                    score = critiques[critic][movie]
                    weight = cosine(critiques[critic], critiques[nouveauCritique])
                    scoreSum += score * weight
                    weightSum += np.abs(weight)

            if weightSum > 0:
                globalScores[movie] = scoreSum / weightSum
    
    globalScores = sorted(globalScores.items(), key=lambda x: x[1], reverse=True)    
    name, score = globalScores[0]
    return (name, round(float(score), 2))

### Helper Functions

In [17]:
def percentageEmptyCells(critiques, movies):
    total_cells = len(movies) * len(critiques)
    empty_cells = 0
    for critic in critiques:
        empty_cells += len(movies) - len(critiques[critic])
    return (empty_cells / total_cells) * 100 if total_cells > 0 else 0

def recommendAll(critic, critiques, movies):
    print("Recommendations for", critic)

    print("Percentage of empty cells:", percentageEmptyCells(critiques, movies), "%")

    print("\nBest Recommend with Nearest Neighbor (Manhattan):")
    print(BestRecommendNearestNeighbor(recommendNearestNeighbor(critic, critiques)))

    print("\nBest Recommend with Manhattan:")
    print(Bestrecommend(critic, critiques, movies))    

    print("\nBest Recommend with Euclidienne:")
    print(BestrecommendEuclidienne(critic, critiques, movies))  

    print("\nBest Recommend with Exponential Weighting:")
    print(BestrecommendwithExp(critic, critiques, movies))

    print("\nPearson Recommend:")
    print(PearsonRecommend(critic, critiques, movies))

    print("\nCosine Recommend:")
    print(CosineRecommend(critic, critiques, movies))

#### Simple Movie Critique

In [18]:
movie_critiques, movies, _ = read_ratings_xlsx("data/movie_critique.xlsx")
recommendAll("Anne", movie_critiques, movies)

Recommendations for Anne
Percentage of empty cells: 20.833333333333336 %

Best Recommend with Nearest Neighbor (Manhattan):
('Night', 4.0)

Best Recommend with Manhattan:
('Superman', 3.91)

Best Recommend with Euclidienne:
('Superman', 3.93)

Best Recommend with Exponential Weighting:
('Night', 3.93)

Pearson Recommend:
('Superman', 1.31)

Cosine Recommend:
('Superman', 3.99)


#### Music Critique



In [19]:
music_critiques, music, _ = read_ratings_xlsx("data/music_critique.xlsx")

In [20]:
recommendAll("Veronica", music_critiques, music)

Recommendations for Veronica
Percentage of empty cells: 23.4375 %

Best Recommend with Nearest Neighbor (Manhattan):
('Broken Bells', 4.0)

Best Recommend with Manhattan:
('Broken Bells', 3.24)

Best Recommend with Euclidienne:
('Broken Bells', 3.12)

Best Recommend with Exponential Weighting:
('Broken Bells', 3.71)

Pearson Recommend:
('Vampire Weekend', 0.28)

Cosine Recommend:
('Broken Bells', 3.02)


In [21]:
recommendAll("Hailey", music_critiques, music)

Recommendations for Hailey
Percentage of empty cells: 23.4375 %

Best Recommend with Nearest Neighbor (Manhattan):
('Phoenix', 4.0)

Best Recommend with Manhattan:
('Phoenix', 4.14)

Best Recommend with Euclidienne:
('Phoenix', 4.18)

Best Recommend with Exponential Weighting:
('Phoenix', 4.13)

Pearson Recommend:
('Phoenix', 4.59)

Cosine Recommend:
('Phoenix', 4.18)


In [22]:
def check_requirements(critiques, movies, df, target_critic="Reviewer1"):
    """
    Check if the dataset satisfies all requirements:
    1. 10 ≤ n reviewers ≤ 20
    2. 10 ≤ m movies ≤ 20
    3. Ratings between 3 and 10
    4. No two reviewers with identical ratings
    5. Target reviewer has seen < 50% of movies
    6. Empty cells between 40% and 70%
    """
    
    print("\n" + "=" * 80)
    print("REQUIREMENTS VERIFICATION")
    print("=" * 80)
    
    results = {}
    
    # 1. Number of reviewers
    n_reviewers = len(critiques)
    req1 = 10 <= n_reviewers <= 20
    results['reviewers'] = req1
    print(f"\n1. Number of reviewers: {n_reviewers}")
    print(f"   Required: 10 ≤ n ≤ 20")
    print(f"   Status: {'✓ PASS' if req1 else '✗ FAIL'}")
    
    # 2. Number of movies
    n_movies = len(movies)
    req2 = 10 <= n_movies <= 20
    results['movies'] = req2
    print(f"\n2. Number of movies: {n_movies}")
    print(f"   Required: 10 ≤ m ≤ 20")
    print(f"   Status: {'✓ PASS' if req2 else '✗ FAIL'}")
    
    # 3. Rating range (3-10)
    all_ratings = []
    for critic in critiques:
        all_ratings.extend(critiques[critic].values())
    min_rating = min(all_ratings)
    max_rating = max(all_ratings)
    req3 = all(3 <= r <= 10 for r in all_ratings)
    results['rating_range'] = req3
    print(f"\n3. Rating range: [{min_rating}, {max_rating}]")
    print(f"   Required: All ratings between 3 and 10")
    print(f"   Status: {'✓ PASS' if req3 else '✗ FAIL'}")
    
    # 4. No identical reviewers
    reviewers_list = list(critiques.keys())
    identical_found = False
    for i in range(len(reviewers_list)):
        for j in range(i + 1, len(reviewers_list)):
            r1 = reviewers_list[i]
            r2 = reviewers_list[j]
            # Compare dictionaries
            if set(critiques[r1].keys()) == set(critiques[r2].keys()):
                if all(critiques[r1].get(k) == critiques[r2].get(k) for k in critiques[r1].keys()):
                    print(f"\n4. Identical reviewers check:")
                    print(f"   ✗ FAIL: {r1} and {r2} have identical ratings")
                    identical_found = True
                    break
        if identical_found:
            break
    
    req4 = not identical_found
    results['no_identical'] = req4
    if not identical_found:
        print(f"\n4. No identical reviewers: ✓ PASS")
    
    # 5. Target reviewer seen < 50%
    if target_critic in critiques:
        movies_seen = len(critiques[target_critic])
        percentage_seen = (movies_seen / n_movies) * 100
        req5 = movies_seen < n_movies / 2
        results['target_seen'] = req5
        print(f"\n5. Target reviewer ({target_critic}) has seen:")
        print(f"   Movies: {movies_seen}/{n_movies} ({percentage_seen:.1f}%)")
        print(f"   Required: Less than 50% (< {n_movies/2})")
        print(f"   Status: {'✓ PASS' if req5 else '✗ FAIL'}")
    else:
        print(f"\n5. Target reviewer ({target_critic}) not found!")
        results['target_seen'] = False
        req5 = False
    
    # 6. Empty cells percentage
    empty_pct = percentageEmptyCells(critiques, movies)
    req6 = 40 <= empty_pct <= 70
    results['empty_cells'] = req6
    print(f"\n6. Empty cells: {empty_pct:.2f}%")
    print(f"   Required: Between 40% and 70%")
    print(f"   Status: {'✓ PASS' if req6 else '✗ FAIL'}")
    
    print(f"\n7. Testing recommendations for {target_critic}...")
    
    methods = {
        "Nearest Neighbor (Manhattan)": lambda: BestRecommendNearestNeighbor(recommendNearestNeighbor(target_critic, critiques)),
        "Manhattan Distance": lambda: Bestrecommend(target_critic, critiques, movies),
        "Euclidean Distance": lambda: BestrecommendEuclidienne(target_critic, critiques, movies),
        "Exponential Weighting": lambda: BestrecommendwithExp(target_critic, critiques, movies),
        "Pearson Correlation": lambda: PearsonRecommend(target_critic, critiques, movies),
        "Cosine Similarity": lambda: CosineRecommend(target_critic, critiques, movies)
    }
    
    recommendations = {}
    for method_name, method_func in methods.items():
        try:
            rec = method_func()
            recommendations[method_name] = rec
            print(f"   {method_name:30s}: {rec[0]:15s} (score: {rec[1]:.2f})" if rec[0] else f"   {method_name:30s}: No recommendation")
        except Exception as e:
            print(f"   {method_name:30s}: Error - {e}")
            recommendations[method_name] = (None, 0)
    

    all_pass = all(results.values())
    
    if all_pass:
        print("ALL REQUIREMENTS SATISFIED")
    else:
        print("SOME REQUIREMENTS NOT SATISFIED:")
        for check, passed in results.items():
            if not passed:
                print(f"   ✗ {check}")
    
    return all_pass, results


#### Movie Critique

In [23]:
print("SAME RECOMMENDATIONS")
critiques, movies, df  = read_ratings_xlsx("data/same_recommendation.xlsx")
check_requirements(critiques, movies, df, target_critic="Reviewer1")


SAME RECOMMENDATIONS

REQUIREMENTS VERIFICATION

1. Number of reviewers: 15
   Required: 10 ≤ n ≤ 20
   Status: ✓ PASS

2. Number of movies: 15
   Required: 10 ≤ m ≤ 20
   Status: ✓ PASS

3. Rating range: [3.0, 10.0]
   Required: All ratings between 3 and 10
   Status: ✓ PASS

4. No identical reviewers: ✓ PASS

5. Target reviewer (Reviewer1) has seen:
   Movies: 5/15 (33.3%)
   Required: Less than 50% (< 7.5)
   Status: ✓ PASS

6. Empty cells: 44.44%
   Required: Between 40% and 70%
   Status: ✓ PASS

7. Testing recommendations for Reviewer1...
   Nearest Neighbor (Manhattan)  : Movie15         (score: 10.00)
   Manhattan Distance            : Movie15         (score: 10.00)
   Euclidean Distance            : Movie15         (score: 10.00)
   Exponential Weighting         : Movie15         (score: 10.00)
   Pearson Correlation           : Movie15         (score: 10.00)
   Cosine Similarity             : Movie15         (score: 10.00)
ALL REQUIREMENTS SATISFIED


(True,
 {'reviewers': True,
  'movies': True,
  'rating_range': True,
  'no_identical': True,
  'target_seen': True,
  'empty_cells': True})

In [ ]:
print("DIFFERENT RECOMMENDATIONS")
critiques, movies, df  = read_ratings_xlsx("data/different_recommendations.xlsx")
check_requirements(critiques, movies, df, target_critic="Reviewer1")

DIFFERENT RECOMMENDATIONS

REQUIREMENTS VERIFICATION

1. Number of reviewers: 16
   Required: 10 ≤ n ≤ 20
   Status: ✓ PASS

2. Number of movies: 16
   Required: 10 ≤ m ≤ 20
   Status: ✓ PASS

3. Rating range: [3.0, 10.0]
   Required: All ratings between 3 and 10
   Status: ✓ PASS

4. No identical reviewers: ✓ PASS

5. Target reviewer (Reviewer1) has seen:
   Movies: 5/16 (31.2%)
   Required: Less than 50% (< 8.0)
   Status: ✓ PASS

6. Empty cells: 59.77%
   Required: Between 40% and 70%
   Status: ✓ PASS

7. Testing recommendations for Reviewer1...
   Nearest Neighbor (Manhattan)  : Movie15         (score: 6.00)
   Manhattan Distance            : Movie7          (score: 5.76)
   Euclidean Distance            : Movie14         (score: 6.07)
   Exponential Weighting         : Movie16         (score: 6.00)
   Pearson Correlation           : Movie12         (score: 5.00)
   Cosine Similarity             : Movie6          (score: 6.05)
ALL REQUIREMENTS SATISFIED


(True,
 {'reviewers': True,
  'movies': True,
  'rating_range': True,
  'no_identical': True,
  'target_seen': True,
  'empty_cells': True})

Method to help in exercise 5, checking the neighboor that constributes the most for each method

In [ ]:
def analyze_contributions(nouveauCritique, critiques, movies):
    print(f"\n{'='*80}")
    print(f"CONTRIBUTION ANALYSIS FOR {nouveauCritique}")
    print(f"{'='*80}\n")
    
    print("1. MANHATTAN DISTANCE METHOD")
    print("-" * 80)
    distances_manhattan = computeNearestNeighbor(nouveauCritique, critiques)
    
    #Get all unwatched movies
    unwatched = [m for m in movies if m not in critiques[nouveauCritique]]
    
    #For each unwatched movie, show contributions
    movie_contributions = {}
    for movie in unwatched:
        contributions = []
        for distance, critic_name in distances_manhattan:
            if movie in critiques[critic_name]:
                weight = 1 / (1 + distance)
                rating = critiques[critic_name][movie]
                contributions.append({
                    'critic': critic_name,
                    'rating': rating,
                    'distance': distance,
                    'weight': weight,
                    'contribution': rating * weight
                })
        movie_contributions[movie] = contributions
    
    #Show top movie's contributions
    top_movie = Bestrecommend(nouveauCritique, critiques, movies)
    print(f"\nTop Recommendation: {top_movie[0]} (Score: {top_movie[1]})")
    print(f"\nContributions from each neighbor:")
    print(f"{'Critic':<20} {'Rating':<8} {'Distance':<10} {'Weight':<10} {'Contribution':<12}")
    print("-" * 80)
    
    contribs = movie_contributions[top_movie[0]]
    total_weight = sum(c['weight'] for c in contribs)
    
    for c in sorted(contribs, key=lambda x: x['contribution'], reverse=True)[:10]:
        print(f"{c['critic']:<20} {c['rating']:<8.1f} {c['distance']:<10.2f} {c['weight']:<10.4f} {c['contribution']:<12.4f}")
    
    print(f"\nWeighted Average = {sum(c['contribution'] for c in contribs) / total_weight:.2f}")
    
    print(f"\n\n2. EUCLIDEAN DISTANCE METHOD")
    print("-" * 80)
    distances_euclidean = computeNearestNeighborEuclidienne(nouveauCritique, critiques)
    
    movie_contributions_euc = {}
    for movie in unwatched:
        contributions = []
        for distance, critic_name in distances_euclidean:
            if movie in critiques[critic_name]:
                weight = 1 / (1 + distance)
                rating = critiques[critic_name][movie]
                contributions.append({
                    'critic': critic_name,
                    'rating': rating,
                    'distance': distance,
                    'weight': weight,
                    'contribution': rating * weight
                })
        movie_contributions_euc[movie] = contributions
    
    top_movie_euc = BestrecommendEuclidienne(nouveauCritique, critiques, movies)
    print(f"\nTop Recommendation: {top_movie_euc[0]} (Score: {top_movie_euc[1]})")
    print(f"\nContributions from each neighbor:")
    print(f"{'Critic':<20} {'Rating':<8} {'Distance':<10} {'Weight':<10} {'Contribution':<12}")
    print("-" * 80)
    
    contribs_euc = movie_contributions_euc[top_movie_euc[0]]
    total_weight_euc = sum(c['weight'] for c in contribs_euc)
    
    for c in sorted(contribs_euc, key=lambda x: x['contribution'], reverse=True)[:10]:
        print(f"{c['critic']:<20} {c['rating']:<8.1f} {c['distance']:<10.2f} {c['weight']:<10.4f} {c['contribution']:<12.4f}")
    
    print(f"\nWeighted Average = {sum(c['contribution'] for c in contribs_euc) / total_weight_euc:.2f}")
    
    print(f"\n\n3. PEARSON CORRELATION METHOD")
    print("-" * 80)
    
    pearson_correlations = []
    for critic in critiques:
        if critic != nouveauCritique:
            corr = pearson(critiques[critic], critiques[nouveauCritique])
            pearson_correlations.append((corr, critic))
    
    movie_contributions_pearson = {}
    for movie in unwatched:
        contributions = []
        for corr, critic_name in pearson_correlations:
            if movie in critiques[critic_name]:
                weight = corr
                rating = critiques[critic_name][movie]
                contributions.append({
                    'critic': critic_name,
                    'rating': rating,
                    'correlation': corr,
                    'weight': np.abs(weight),
                    'contribution': rating * weight
                })
        movie_contributions_pearson[movie] = contributions
    
    top_movie_pearson = PearsonRecommend(nouveauCritique, critiques, movies)
    print(f"\nTop Recommendation: {top_movie_pearson[0]} (Score: {top_movie_pearson[1]})")
    print(f"\nContributions from each neighbor:")
    print(f"{'Critic':<20} {'Rating':<8} {'Correlation':<12} {'|Weight|':<10} {'Contribution':<12}")
    print("-" * 80)
    
    contribs_pearson = movie_contributions_pearson[top_movie_pearson[0]]
    total_weight_pearson = sum(c['weight'] for c in contribs_pearson)
    
    for c in sorted(contribs_pearson, key=lambda x: abs(x['contribution']), reverse=True)[:10]:
        print(f"{c['critic']:<20} {c['rating']:<8.1f} {c['correlation']:<12.4f} {c['weight']:<10.4f} {c['contribution']:<12.4f}")
    
    print(f"\nWeighted Average = {sum(c['contribution'] for c in contribs_pearson) / total_weight_pearson:.2f}")
    
    print(f"\n\n4. COSINE SIMILARITY METHOD")
    print("-" * 80)
    
    cosine_similarities = []
    for critic in critiques:
        if critic != nouveauCritique:
            sim = cosine(critiques[critic], critiques[nouveauCritique])
            cosine_similarities.append((sim, critic))
    
    movie_contributions_cosine = {}
    for movie in unwatched:
        contributions = []
        for sim, critic_name in cosine_similarities:
            if movie in critiques[critic_name]:
                weight = sim
                rating = critiques[critic_name][movie]
                contributions.append({
                    'critic': critic_name,
                    'rating': rating,
                    'similarity': sim,
                    'weight': np.abs(weight),
                    'contribution': rating * weight
                })
        movie_contributions_cosine[movie] = contributions
    
    top_movie_cosine = CosineRecommend(nouveauCritique, critiques, movies)
    print(f"\nTop Recommendation: {top_movie_cosine[0]} (Score: {top_movie_cosine[1]})")
    print(f"\nContributions from each neighbor:")
    print(f"{'Critic':<20} {'Rating':<8} {'Similarity':<12} {'|Weight|':<10} {'Contribution':<12}")
    print("-" * 80)
    
    contribs_cosine = movie_contributions_cosine[top_movie_cosine[0]]
    total_weight_cosine = sum(c['weight'] for c in contribs_cosine)
    
    for c in sorted(contribs_cosine, key=lambda x: abs(x['contribution']), reverse=True)[:10]:
        print(f"{c['critic']:<20} {c['rating']:<8.1f} {c['similarity']:<12.4f} {c['weight']:<10.4f} {c['contribution']:<12.4f}")
    
    print(f"\nWeighted Average = {sum(c['contribution'] for c in contribs_cosine) / total_weight_cosine:.2f}")


critiques, movies, df = read_ratings_xlsx("data/different_recommendations.xlsx")
analyze_contributions("Reviewer1", critiques, movies)


CONTRIBUTION ANALYSIS FOR Reviewer1

1. MANHATTAN DISTANCE METHOD
--------------------------------------------------------------------------------

Top Recommendation: Movie7 (Score: 5.76)

Contributions from each neighbor:
Critic               Rating   Distance   Weight     Contribution
--------------------------------------------------------------------------------
Reviewer16           6.0      2.00       0.3333     2.0000      
Reviewer5            6.0      3.00       0.2500     1.5000      
Reviewer3            6.0      8.00       0.1111     0.6667      
Reviewer9            6.0      13.00      0.0714     0.4286      
Reviewer10           6.0      16.00      0.0588     0.3529      
Reviewer15           6.0      16.00      0.0588     0.3529      
Reviewer8            3.0      12.00      0.0769     0.2308      

Weighted Average = 5.76


2. EUCLIDEAN DISTANCE METHOD
--------------------------------------------------------------------------------

Top Recommendation: Movie14 (Score: 